# Correlation matrix and visualization of variables and GDP

### This is the master notebook, please copy and create a new one for each group.

In [16]:
import numpy as np
from scipy import stats
from scipy.stats import shapiro
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
pip install matplotlib


  Using cached matplotlib-3.5.2-cp310-cp310-win_amd64.whl (7.2 MB)
     ---------------------------------------- 3.3/3.3 MB 1.3 MB/s eta 0:00:00
  Using cached fonttools-4.33.3-py3-none-any.whl (930 kB)
     -------------------------------------- 55.3/55.3 KB 714.8 kB/s eta 0:00:00
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)


You should consider upgrading via the 'c:\Users\mferna38\Documents\GitHub\python-data-driven-decisions\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [18]:
import matplotlib.pyplot as plt

In [5]:
df= pd.read_csv (os.getcwd()+'\\Data'+'\\Latam.csv')
df

,Unnamed: 0,Country,Year,Alcohol per capita,Education GExp,Employment-agriculture,Employment-industry,Employment-services,Exports-Commercial services,Exports-G&S,...,Literacy rate,Mortality-infants,Mortality-pollution,Net migration,Ninis,R&D GExp,Renewable electricity,Suicide,Workers high education,Continent
0,1193,SEN,1990,1.00000,0.299902,1.000000,0.987341,0.000000,0.093464,0.056127,...,0.000000,0.649822,NaN,1.000000,0.678788,0.000000,0.100,1.000000,0.202163,South Africa
1,1194,SEN,1991,1.00000,0.299902,1.000000,0.987341,0.000000,0.030230,0.027220,...,0.000000,0.669587,NaN,1.000000,0.678788,0.000000,0.100,1.000000,0.202163,South Africa
2,1195,SEN,1992,1.00000,0.299902,0.996279,0.873418,0.008511,0.071855,0.035561,...,0.000000,0.707387,NaN,1.000000,0.678788,0.000000,0.125,1.000000,0.202163,South Africa
3,1196,SEN,1993,1.00000,0.299902,0.990962,0.772152,0.018617,0.000000,0.000000,...,0.000000,0.758227,NaN,0.800000,0.678788,0.000000,0.125,1.000000,0.202163,South Africa
4,1197,SEN,1994,1.00000,0.299902,0.985646,0.708861,0.026596,0.043933,0.014946,...,0.000000,0.816403,NaN,0.600000,0.678788,0.000000,0.125,1.000000,0.202163,South Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3548,247,CHN,2016,0.99002,0.246406,0.068956,0.831461,0.867725,0.849888,0.804316,...,0.977482,0.042573,NaN,0.264203,NaN,0.974482,1.000,0.016129,NaN,Pair
3549,248,CHN,2017,0.98004,0.212024,0.048007,0.753933,0.917460,0.867490,0.888169,...,0.988741,0.030745,NaN,0.243061,NaN,0.984436,1.000,0.000000,NaN,Pair
3550,249,CHN,2018,0.97006,0.000000,0.021530,0.777528,0.942152,0.953748,0.974699,...,1.000000,0.019650,NaN,0.243061,NaN,1.000000,1.000,0.000000,NaN,Pair
3551,250,CHN,2019,0.97006,0.000000,0.000000,0.676405,1.000000,1.000000,0.964730,...,1.000000,0.009383,NaN,0.243061,NaN,1.000000,1.000,0.000000,NaN,Pair


## Visualization

In [6]:
columns=['Country','Year','Gender equality','Exports-Commercial services','Renewable electricity','Employment-agriculture','Employment-industry','Employment-services','Exports-G&S','Fertility rate','Foreign investment','GDP','Education GExp','Workers high education','Literacy rate','Mortality-pollution','Net migration','Mortality-infants','Health services use','R&D GExp','Ninis','Suicide','International taxes','Alcohol per capita']
clist=df['Country'].unique()

The following loop has been created to test if each variable for each country follows a normal distribution and also to present in an orderly manner all the correlations between the GDP and the variables so they can be understood on a lower scale.

In [20]:
def multcolumn(frame):
    for u in range(2, len(columns)):
        name2=columns[u]+'^2'
        name3=columns[u]+'^3'
        namelog=columns[u]+' log'
        frame.loc[:,name2] = frame[columns[u]]**2
        frame.loc[:,name3] = frame[columns[u]]**3
        frame.loc[:,namelog] = np.log(frame[columns[u]])

In [28]:
multcolumn(df)
for i in range(0,len(clist)):
    dat=df.loc[df.loc[:, 'Country'] == clist[i]]
    for e in range(2,len(columns)):
        data=dat.iloc[:, e]
        stat, p = shapiro(data)
        print(clist[i] +"-"+ columns[e])
        print('Statistical=%.3f, p=%.3f' % (stat, p))
        alpha = 0.05
        if p > alpha:
            print('Data is NORMAL ( H0 not denied )')
        else:
            print('It does NOT seem normal( Denies H0 )')
    print(clist[i])
    cor=dat.corr()
    cor.loc[:,'GDP-R^2']=cor['GDP']**2   
    corcolumn=cor[['GDP-R^2','GDP']].sort_values(by = 'GDP-R^2',ascending = False).style.background_gradient()
    display(corcolumn)

SEN-Gender equality
Statistical=0.952, p=0.000
It does NOT seem normal( Denies H0 )
SEN-Exports-Commercial services
Statistical=0.856, p=0.000
It does NOT seem normal( Denies H0 )
SEN-Renewable electricity
Statistical=0.931, p=0.000
It does NOT seem normal( Denies H0 )
SEN-Employment-agriculture
Statistical=0.889, p=0.000
It does NOT seem normal( Denies H0 )
SEN-Employment-industry
Statistical=0.933, p=0.000
It does NOT seem normal( Denies H0 )
SEN-Employment-services
Statistical=0.901, p=0.000
It does NOT seem normal( Denies H0 )
SEN-Exports-G&S
Statistical=0.816, p=0.000
It does NOT seem normal( Denies H0 )
SEN-Fertility rate
Statistical=0.861, p=0.000
It does NOT seem normal( Denies H0 )
SEN-Foreign investment
Statistical=0.936, p=0.000
It does NOT seem normal( Denies H0 )
SEN-GDP
Statistical=0.880, p=0.000
It does NOT seem normal( Denies H0 )
SEN-Education GExp
Statistical=0.875, p=0.000
It does NOT seem normal( Denies H0 )
SEN-Workers high education
Statistical=nan, p=1.000
Data i

c:\Users\mferna38\Documents\GitHub\python-data-driven-decisions\.venv\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


ImportError: background_gradient requires matplotlib.

ZAF-Gender equality
Statistical=0.952, p=0.000
It does NOT seem normal( Denies H0 )
ZAF-Exports-Commercial services
Statistical=0.836, p=0.000
It does NOT seem normal( Denies H0 )
ZAF-Renewable electricity
Statistical=0.837, p=0.000
It does NOT seem normal( Denies H0 )
ZAF-Employment-agriculture
Statistical=0.830, p=0.000
It does NOT seem normal( Denies H0 )
ZAF-Employment-industry
Statistical=0.906, p=0.000
It does NOT seem normal( Denies H0 )
ZAF-Employment-services
Statistical=0.868, p=0.000
It does NOT seem normal( Denies H0 )
ZAF-Exports-G&S
Statistical=0.863, p=0.000
It does NOT seem normal( Denies H0 )
ZAF-Fertility rate
Statistical=0.857, p=0.000
It does NOT seem normal( Denies H0 )
ZAF-Foreign investment
Statistical=0.830, p=0.000
It does NOT seem normal( Denies H0 )
ZAF-GDP
Statistical=0.962, p=0.000
It does NOT seem normal( Denies H0 )
ZAF-Education GExp
Statistical=0.882, p=0.000
It does NOT seem normal( Denies H0 )
ZAF-Workers high education
Statistical=nan, p=1.000
Data i

ImportError: background_gradient requires matplotlib.

...

ImportError: background_gradient requires matplotlib.

USA-Gender equality
Statistical=0.953, p=0.017
It does NOT seem normal( Denies H0 )
USA-Exports-Commercial services
Statistical=0.828, p=0.000
It does NOT seem normal( Denies H0 )
USA-Renewable electricity
Statistical=nan, p=1.000
Data is NORMAL ( H0 not denied )
USA-Employment-agriculture
Statistical=0.850, p=0.000
It does NOT seem normal( Denies H0 )
USA-Employment-industry
Statistical=0.834, p=0.000
It does NOT seem normal( Denies H0 )
USA-Employment-services
Statistical=0.831, p=0.000
It does NOT seem normal( Denies H0 )
USA-Exports-G&S
Statistical=0.898, p=0.000
It does NOT seem normal( Denies H0 )
USA-Fertility rate
Statistical=0.897, p=0.000
It does NOT seem normal( Denies H0 )
USA-Foreign investment
Statistical=0.886, p=0.000
It does NOT seem normal( Denies H0 )
USA-GDP
Statistical=0.949, p=0.010
It does NOT seem normal( Denies H0 )
USA-Education GExp
Statistical=0.940, p=0.004
It does NOT seem normal( Denies H0 )
USA-Workers high education
Statistical=nan, p=1.000
Data is NORM

ImportError: background_gradient requires matplotlib.

CHN-Gender equality
Statistical=0.953, p=0.019
It does NOT seem normal( Denies H0 )
CHN-Exports-Commercial services
Statistical=0.715, p=0.000
It does NOT seem normal( Denies H0 )
CHN-Renewable electricity
Statistical=0.926, p=0.001
It does NOT seem normal( Denies H0 )
CHN-Employment-agriculture
Statistical=0.919, p=0.001
It does NOT seem normal( Denies H0 )
CHN-Employment-industry
Statistical=0.875, p=0.000
It does NOT seem normal( Denies H0 )
CHN-Employment-services
Statistical=0.950, p=0.014
It does NOT seem normal( Denies H0 )
CHN-Exports-G&S
Statistical=0.894, p=0.000
It does NOT seem normal( Denies H0 )
CHN-Fertility rate
Statistical=0.837, p=0.000
It does NOT seem normal( Denies H0 )
CHN-Foreign investment
Statistical=0.873, p=0.000
It does NOT seem normal( Denies H0 )
CHN-GDP
Statistical=0.928, p=0.001
It does NOT seem normal( Denies H0 )
CHN-Education GExp
Statistical=0.821, p=0.000
It does NOT seem normal( Denies H0 )
CHN-Workers high education
Statistical=nan, p=1.000
Data i

ImportError: background_gradient requires matplotlib.